In [3]:
import textagon
import pandas as pd

df = pd.read_csv('dvd.txt', sep='\t', header=None, names=["classLabels", "corpus"])




In [4]:
df.head()

,classLabels,corpus
0,1,"I expected so much more from helo this movie, ..."
1,1,I am amazed and mystified that most of the rev...
2,A,I suppose it's alright if you like overlong mu...
3,1,"The movie is a about a guy, who does a lot of ..."
4,1,Maybe it's because I had never read the book a...
